In [53]:
# How to incorporate model checking wAIC into this??
# Why didn't the FINEMAP people do any standard model checking? Can we improve this?

import numpy as np
from sklearn.preprocessing import scale
import pandas as pd
from scipy.stats import multivariate_normal

In [30]:
X = np.array([[1, 1], [1, 1], [1, 1], [1, 0], [0, 0], [0, 0], [0, 0], [0, 1]])
X = scale(X)
X

/Users/bdoughty/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


array([[ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1., -1.],
       [-1., -1.],
       [-1., -1.],
       [-1., -1.],
       [-1.,  1.]])

In [31]:
Y = np.array([1, 1, 1, 1, 0, 0, 0, 0])
Y = scale(Y)
Y

/Users/bdoughty/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


array([ 1.,  1.,  1.,  1., -1., -1., -1., -1.])

In [32]:
X.T @ X

array([[ 8.,  4.],
       [ 4.,  8.]])

In [33]:
X.T @ Y

array([ 8.,  4.])

In [34]:
B = np.array([1, 0])
X @ B

array([ 1.,  1.,  1.,  1., -1., -1., -1., -1.])

In [35]:
X.T @ X @ B

array([ 8.,  4.])

In [116]:
# run finemap on the two loci that benner gives
# need z, R, and finemap post probs
finemapdir = '/Users/bdoughty/Downloads/finemap_v1.1_MacOSX/example/'
ld = np.asmatrix(pd.read_csv(finemapdir + "region1.ld", delim_whitespace=True, header=None), dtype=np.float32)
z = pd.read_csv(finemapdir + "region1.z", delim_whitespace=True, header=None).as_matrix(dtype=np.float32)[:,1]
pp = pd.read_csv(finemapdir + "region1.snp", delim_whitespace=True).sort_values('index').as_matrix(dtype=np.float32)[:,2]

n = 5363
s_sq = 0.05 ** 2
sigma_sq = 1 ** 2

xty = np.sqrt(n) * z
xtx = n * ld

# create distribution for lambda from posterior by plugging into formula
def sample_lambda(pp, size=1):
    Deltay = np.diag(pp)
    cov = Deltay * s_sq * sigma_sq
    return multivariate_normal.rvs(mean=None, cov=cov, size=size)

# evaluate the expectation of || z - R lambda || wrt p(lambda) by sampling
#xty - xtx sample_lambda(pp)
# can also try using posterior predictive for p(lambda) = int dgamma p(lambda | gamma) p(gamma | X)x

TypeError: as_matrix() got an unexpected keyword argument 'dtype'

In [86]:
np.linalg.norm(xty - xtx @ sample_lambda(pp))

1683.9866955315736

In [114]:
np.linalg.norm((np.asmatrix(xty).T - xtx @ sample_lambda(pp, 100).T), axis=1)

AttributeError: 'float' object has no attribute 'sqrt'

In [101]:
np.asmatrix(xty).T.shape

(50, 1)